In [169]:
import geopandas as gpd
import pandas as pd

In [170]:
world = gpd.read_file('../data/world_countries.geojson')
df = pd.read_csv('../data/clean_data.csv')

In [171]:
df = df.drop(columns=['Unnamed: 0'])
world = world.drop(columns=['iso_a3', 'gdp_md_est', 'pop_est'])

In [172]:
df['country'] = df['country'].str.split(', ')
df = df.explode('country').reset_index(drop=True)
df = df[df['country'] != '']

countries = list(set(df['country'].to_list()))
countries.remove('worldwide')

df['country'].replace(to_replace='worldwide',value=', '.join(countries), inplace=True)
df['country'] = df['country'].str.split(', ')
df = df.explode('country').reset_index(drop=True)

In [173]:
df.to_csv('./data.csv')

In [174]:
df['count'] = 1
pivot_df = df.pivot_table(index='country', columns='type', values='count', aggfunc='sum', fill_value=0).reset_index()

In [175]:
pivot_df

type,country,Movie,TV Show
0,Afghanistan,8437,2885
1,Albania,8436,2885
2,Algeria,8437,2885
3,Angola,8436,2885
4,Argentina,8506,2909
...,...,...,...
126,Vatican City,8435,2885
127,Venezuela,8438,2885
128,Vietnam,8441,2885
129,West Germany,8437,2887


In [176]:
world = pd.merge(world, pivot_df, how='left', left_on='name', right_on='country').drop(columns=['country'])

In [177]:
world.fillna(0, inplace=True)

In [178]:
world

,continent,name,geometry,Movie,TV Show
0,Oceania,Fiji,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",0.0,0.0
1,Africa,Tanzania,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",8435.0,2885.0
2,Africa,W. Sahara,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",0.0,0.0
3,North America,Canada,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",8877.0,3065.0
4,North America,United States of America,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",0.0,0.0
...,...,...,...,...,...
172,Europe,Serbia,"POLYGON ((18.82982 45.90887, 18.82984 45.90888...",8442.0,2885.0
173,Europe,Montenegro,"POLYGON ((20.07070 42.58863, 19.80161 42.50009...",8435.0,2885.0
174,Europe,Kosovo,"POLYGON ((20.59025 41.85541, 20.52295 42.21787...",8436.0,2885.0
175,North America,Trinidad and Tobago,"POLYGON ((-61.68000 10.76000, -61.10500 10.890...",0.0,0.0


In [108]:
output_geojson_path = './world.geojson'
world.to_file(output_geojson_path, driver='GeoJSON')